In [1]:
!pip3 install --upgrade pip
!pip install setuptools==57.5.0 #adde
!pip3 install dccp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import traceback
import dccp

from copy import deepcopy

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import cvxpy
import sys
import os
print(os.getcwd())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.1
    Uninstalling setuptools-67.6.1:
      Successfully uninstalled setuptools-67.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.1 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setu

In [2]:
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
origin_df = pd.read_csv(url)
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       7214 non-null   int64  
 1   name                     7214 non-null   object 
 2   first                    7214 non-null   object 
 3   last                     7214 non-null   object 
 4   compas_screening_date    7214 non-null   object 
 5   sex                      7214 non-null   object 
 6   dob                      7214 non-null   object 
 7   age                      7214 non-null   int64  
 8   age_cat                  7214 non-null   object 
 9   race                     7214 non-null   object 
 10  juv_fel_count            7214 non-null   int64  
 11  decile_score             7214 non-null   int64  
 12  juv_misd_count           7214 non-null   int64  
 13  juv_other_count          7214 non-null   int64  
 14  priors_count            

In [3]:
df = origin_df.loc[:,["age_cat","sex","race","priors_count","c_charge_degree","two_year_recid"]].query('race in ["African-American","Caucasian"]')
df.reset_index(drop=True,inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6150 entries, 0 to 6149
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age_cat          6150 non-null   object
 1   sex              6150 non-null   object
 2   race             6150 non-null   object
 3   priors_count     6150 non-null   int64 
 4   c_charge_degree  6150 non-null   object
 5   two_year_recid   6150 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 288.4+ KB


In [4]:
features = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"]
cont_features = ["priors_count"]
predicted_feature = "two_year_recid"
sensitive_feature = "race"

In [5]:
def data_preprocessing(df):
    df = df.dropna(subset=["days_b_screening_arrest"]).loc[:,features + [predicted_feature]].query('race in ["African-American","Caucasian"]')

    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])

    y = data[predicted_feature]
    y[y==0] = -1


    X = np.array([]).reshape(len(y), 0) # empty array with num rows same as num examples, will hstack the features to it
    x_sensitive_feature = None

    feature_names = []
    for feature in features:
        vals = data[feature]
        if feature in cont_features:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals) 
            vals = np.reshape(vals, (len(y), -1)) 

        else: 
            lb = preprocessing.LabelBinarizer()
            lb.fit(vals)
            vals = lb.transform(vals)

        if feature == sensitive_feature:
            x_sensitive_feature = vals


        X = np.hstack((X, vals))

        if feature in cont_features: 
            feature_names.append(feature)
        else: 
            if vals.shape[1] == 1: 
                feature_names.append(feature)
            else:
                for k in lb.classes_: 
                    feature_names.append(feature + "_" + str(k))

    x_sensitive_feature = np.array(x_sensitive_feature).flatten()
    X = np.concatenate((np.ones(X.shape[0]).reshape(X.shape[0], 1), X), axis = 1)

    feature_names = ["intercept"] + feature_names
    print(f"Features we will be using for classification are: {feature_names}")


    return X, y, x_sensitive_feature

In [6]:
X,y,x_race = data_preprocessing(origin_df)
print(X,y,x_race)

Features we will be using for classification are: ['intercept', 'age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree']
[[ 1.          1.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          0.          0.         ...  1.          0.05593295
   0.        ]
 [ 1.          1.          0.         ...  1.          2.02993903
   0.        ]
 ...
 [ 1.          0.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          0.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          1.          0.         ...  0.         -0.14146765
   1.        ]] [ 1  1  1 ... -1 -1 -1] [0 0 1 ... 0 0 0]


In [7]:
X_train,X_test,y_train,y_test,x_race_train,x_race_test = train_test_split(X,y,x_race,test_size=1/7,random_state=5243)
print(X_train.shape,X_test.shape,X.shape)


(5070, 8) (845, 8) (5915, 8)


In [8]:
def get_distance_boundary(w, x, s_attr_arr):
    distances_boundary = np.zeros(x.shape[0])
    if isinstance(w, dict): 
        for k in w.keys(): 
            d = np.dot(x, w[k])
            distances_boundary[s_attr_arr == k] = d[s_attr_arr == k] 
    else: 
        distances_boundary = np.dot(x, w)
    return distances_boundary

In [9]:
def get_one_hot_encoding(in_arr):
    in_arr = np.array(in_arr, dtype=int)
    attr_vals_uniq_sorted = sorted(list(set(in_arr)))
    num_uniq_vals = len(attr_vals_uniq_sorted)
    if (num_uniq_vals == 2) and (attr_vals_uniq_sorted[0] == 0 and attr_vals_uniq_sorted[1] == 1):
        return in_arr, None

    
    index_dict = {}
    for i in range(0,len(attr_vals_uniq_sorted)):
        val = attr_vals_uniq_sorted[i]
        index_dict[val] = i

    out_arr = []    
    for i in range(0,len(in_arr)):
        tup = np.zeros(num_uniq_vals)
        val = in_arr[i]
        ind = index_dict[val]
        tup[ind] = 1 # set that value of tuple to 1
        out_arr.append(tup)

    return np.array(out_arr), index_dict

In [10]:
def get_constraint_list_cov(x_train, y_train, x_control_train, sensitive_attrs_to_cov_thresh, cons_type, w):

    """
    get the list of constraints to be fed to the minimizer
    cons_type == 0: means the whole combined misclassification constraint (without FNR or FPR)
    cons_type == 1: FPR constraint
    cons_type == 2: FNR constraint
    cons_type == 4: both FPR as well as FNR constraints
    sensitive_attrs_to_cov_thresh: is a dict like {s: {cov_type: val}}
    s is the sensitive attr
    cov_type is the covariance type. contains the covariance for all misclassifications, FPR and for FNR etc
    """

    constraints = []

    attr_arr = x_control_train
    attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)
            
    if index_dict is None: # binary attribute, in this case, the attr_arr_transformed is the same as the attr_arr

        s_val_to_total = {ct:{} for ct in [0,1,2]} # constrain type -> sens_attr_val -> total number
        s_val_to_avg = {ct:{} for ct in [0,1,2]}
        cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

        for v in set(attr_arr):
            s_val_to_total[0][v] = sum(x_control_train == v)
            s_val_to_total[1][v] = sum(np.logical_and(x_control_train == v, y_train == -1)) # FPR constraint so we only consider the ground truth negative dataset for computing the covariance
            s_val_to_total[2][v] = sum(np.logical_and(x_control_train == v, y_train == +1))


        for ct in [0,1,2]:
            s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1/N in our formulation, differs from one constraint type to another
            s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0/N

        
        for v in set(attr_arr):

            idx = x_control_train == v                

            dist_bound_prod = cvxpy.multiply(y_train[idx], x_train[idx] * w) # y.f(x)
            
            cons_sum_dict[0][v] = cvxpy.sum( cvxpy.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(x_train)) # avg misclassification distance from boundary
            cons_sum_dict[1][v] = cvxpy.sum( cvxpy.minimum(0, cvxpy.multiply( (1 - y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[1][v] / sum(y_train == -1)) # avg false positive distance from boundary (only operates on the ground truth neg dataset)
            cons_sum_dict[2][v] = cvxpy.sum( cvxpy.minimum(0, cvxpy.multiply( (1 + y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[2][v] / sum(y_train == +1)) # avg false negative distance from boundary

            
        if cons_type == 4:
            cts = [1,2]
        elif cons_type in [0,1,2]:
            cts = [cons_type]
        
        else:
            raise Exception("Invalid constraint type")


        for ct in cts:
            thresh = abs(sensitive_attrs_to_cov_thresh[ct][1] - sensitive_attrs_to_cov_thresh[ct][0])
            constraints.append( cons_sum_dict[ct][1] <= cons_sum_dict[ct][0]  + thresh )
            constraints.append( cons_sum_dict[ct][1] >= cons_sum_dict[ct][0]  - thresh )


    return constraints

In [11]:
def train_model(X,y,x_sensitive_feature,eps,constraint_params=None):
    max_iters = 100 
    max_iter_dccp = 50 

    
    num_points, num_features = X.shape
    w = cvxpy.Variable(num_features)

    np.random.seed(5243)
    w.value = np.random.rand(X.shape[1])

    loss = cvxpy.sum(cvxpy.logistic(cvxpy.multiply(-y, X*w) )  ) / num_points 

    if constraint_params is None: # just train a simple classifier, no fairness constraints
        constraints = []
    else:
        constraints = get_constraint_list_cov(X, y, x_sensitive_feature, constraint_params["sensitive_attrs_to_cov_thresh"], constraint_params["cons_type"], w)
    p = cvxpy.Problem(cvxpy.Minimize(loss), [])
    p.solve()

    prob = cvxpy.Problem(cvxpy.Minimize(loss), constraints)
    try:
        tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset
        if constraint_params is not None: # in case we passed these parameters as a part of dccp constraints
            if constraint_params.get("tau") is not None: tau = constraint_params["tau"]
            if constraint_params.get("mu") is not None: mu = constraint_params["mu"]

        prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver=cvxpy.ECOS, verbose=False, 
            feastol=eps, abstol=eps, reltol=eps,feastol_inacc=eps, abstol_inacc=eps, reltol_inacc=eps,
            max_iters=max_iters, max_iter=max_iter_dccp)

        
        assert(prob.status == "Converged" or prob.status == "optimal")

    except:
        traceback.print_exc()
        sys.stdout.flush()
        sys.exit(1)
    w = np.array(w.value).flatten() 

    return w

In [12]:
def get_clf_stats(w, x_train, y_train, x_control_train, x_test, y_test, x_control_test):


    # compute distance from boundary
    distances_boundary_train = get_distance_boundary(w, x_train, x_control_train)
    distances_boundary_test = get_distance_boundary(w, x_test, x_control_test)

    # compute the class labels
    all_class_labels_assigned_train = np.sign(distances_boundary_train)
    all_class_labels_assigned_test = np.sign(distances_boundary_test)


    train_score, test_score, correct_answers_train, correct_answers_test = check_accuracy(None, x_train, y_train, x_test, y_test, all_class_labels_assigned_train, all_class_labels_assigned_test)

  
        
    print_stats = False 
    s_attr_to_fp_fn_train = get_fpr_fnr_sensitive_features(y_train, all_class_labels_assigned_train, x_control_train, print_stats)
    cov_all_train = get_sensitive_attr_constraint_fpr_fnr_cov(None, x_train, y_train, distances_boundary_train, x_control_train) 
    

    print("\n")
    print(f"Accuracy: {test_score}")
    print_stats = True # only print stats for the test fold
    s_attr_to_fp_fn_test = get_fpr_fnr_sensitive_features(y_test, all_class_labels_assigned_test, x_control_test, print_stats)
    cov_all_test = get_sensitive_attr_constraint_fpr_fnr_cov(None, x_test, y_test, distances_boundary_test, x_control_test) 
    print("\n")

    return train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test

In [13]:
def get_fpr_fnr_sensitive_features(y_true, y_pred, x_control, verbose = False):



    # we will make some changes to x_control in this function, so make a copy in order to preserve the origianl referenced object
    x_control_internal = deepcopy(x_control)

    s_attr_to_fp_fn = {}
    
    s_attr_vals = x_control_internal
    for s_val in sorted(list(set(s_attr_vals))):
        s_attr_to_fp_fn[s_val] = {}
        y_true_local = y_true[s_attr_vals==s_val]
        y_pred_local = y_pred[s_attr_vals==s_val]

        

        acc = float(sum(y_true_local==y_pred_local)) / len(y_true_local)

        fp = sum(np.logical_and(y_true_local == -1.0, y_pred_local == +1.0)) # something which is -ve but is misclassified as +ve
        fn = sum(np.logical_and(y_true_local == +1.0, y_pred_local == -1.0)) # something which is +ve but is misclassified as -ve
        tp = sum(np.logical_and(y_true_local == +1.0, y_pred_local == +1.0)) # something which is +ve AND is correctly classified as +ve
        tn = sum(np.logical_and(y_true_local == -1.0, y_pred_local == -1.0)) # something which is -ve AND is correctly classified as -ve

        all_neg = sum(y_true_local == -1.0)
        all_pos = sum(y_true_local == +1.0)

        fpr = float(fp) / float(fp + tn)
        fnr = float(fn) / float(fn + tp)
        tpr = float(tp) / float(tp + fn)
        tnr = float(tn) / float(tn + fp)


        s_attr_to_fp_fn[s_val]["fp"] = fp
        s_attr_to_fp_fn[s_val]["fn"] = fn
        s_attr_to_fp_fn[s_val]["fpr"] = fpr
        s_attr_to_fp_fn[s_val]["fnr"] = fnr

        s_attr_to_fp_fn[s_val]["acc"] = (tp + tn) / (tp + tn + fp + fn)

    return s_attr_to_fp_fn

In [14]:
def get_sensitive_attr_constraint_fpr_fnr_cov(model, x_arr, y_arr_true, y_arr_dist_boundary, x_control_arr, verbose=False):
        
    assert(x_arr.shape[0] == x_control_arr.shape[0])
    if len(x_control_arr.shape) > 1: # make sure we just have one column in the array
        assert(x_control_arr.shape[1] == 1)
    if len(set(x_control_arr)) != 2: # non binary attr
        raise Exception("Non binary attr, fix to handle non bin attrs")

    
    arr = []
    if model is None:
        arr = y_arr_dist_boundary * y_arr_true # simply the output labels
    else:
        arr = np.dot(model, x_arr.T) * y_arr_true # the product with the weight vector -- the sign of this is the output label
    arr = np.array(arr)

    s_val_to_total = {ct:{} for ct in [0,1,2]}
    s_val_to_avg = {ct:{} for ct in [0,1,2]}
    cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

    for v in set(x_control_arr):
        s_val_to_total[0][v] = sum(x_control_arr == v)
        s_val_to_total[1][v] = sum(np.logical_and(x_control_arr == v, y_arr_true == -1))
        s_val_to_total[2][v] = sum(np.logical_and(x_control_arr == v, y_arr_true == +1))


    for ct in [0,1,2]:
        s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1 / N
        s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0 / N

    
    for v in set(x_control_arr):
        idx = x_control_arr == v
        dist_bound_prod = arr[idx]

        cons_sum_dict[0][v] = sum( np.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(x_arr))
        cons_sum_dict[1][v] = sum( np.minimum(0, ( (1 - y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[1][v] / sum(y_arr_true == -1))
        cons_sum_dict[2][v] = sum( np.minimum(0, ( (1 + y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[2][v] / sum(y_arr_true == +1))
        

    cons_type_to_name = {0:"ALL", 1:"FPR", 2:"FNR"}
    for cons_type in [0,1,2]:
        cov_type_name = cons_type_to_name[cons_type]    
        cov = cons_sum_dict[cons_type][1] - cons_sum_dict[cons_type][0]
        
    return cons_sum_dict

In [15]:
def check_accuracy(model, x_train, y_train, x_test, y_test, y_train_predicted, y_test_predicted):
    if model is not None and y_test_predicted is not None:
        print("Either the model (w) or the predicted labels should be None")
        raise Exception("Either the model (w) or the predicted labels should be None")

    if model is not None:
        y_test_predicted = np.sign(np.dot(x_test, model))
        y_train_predicted = np.sign(np.dot(x_train, model))

    def get_accuracy(y, Y_predicted):
        correct_answers = (Y_predicted == y).astype(int) # will have 1 when the prediction and the actual label match
        accuracy = float(sum(correct_answers)) / float(len(correct_answers))
        return accuracy, sum(correct_answers)

    train_score, correct_answers_train = get_accuracy(y_train, y_train_predicted)
    test_score, correct_answers_test = get_accuracy(y_test, y_test_predicted)

    return train_score, test_score, correct_answers_train, correct_answers_test

In [16]:
def train():
    w = train_model(X_train,y_train,x_race_train,eps,constraint_params)
    train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = get_clf_stats(w, X_train, y_train, x_race_train, X_test, y_test, x_race_test)
    return w, test_score, s_attr_to_fp_fn_test

In [ ]:
train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = get_clf_stats(w, X_train, y_train, x_race_train, X_test, y_test, x_race_test, sensitive_attrs)


NameError: ignored

In [17]:
eps = 1e-6
sensitive_attrs_to_cov_thresh = {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}} # zero covariance threshold, means try to get the fairest solution
tau = 5.0
mu = 1.2
idx_to_constraint = {-1:"No Constraint",0:"Misclassification",1:"Only FPR",2:"Only FNR",4:"Both FPR and FNR"}
summary_W = pd.DataFrame(columns=["Constraint_type","Accuracy","FPR","FNR"])
summary_B = pd.DataFrame(columns=["Constraint_type","Accuracy","FPR","FNR"])
summary_diff = pd.DataFrame(columns=["Constraint_type","Overall Accuracy","Diff_Accuracy","Diff_FPR","Diff_FNR"])

In [18]:
for i in [-1,0,1,2,4]:
    if i == -1:
      constraint_params = None
    else:
      constraint_params = {"cons_type": i, 
              "tau": tau, 
              "mu": mu, 
              "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}
    w, test_score, s_attr_to_fp_fn_test = train()
    summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
    summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
    summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"Overall Accuracy":test_score,"Diff_Accuracy":s_attr_to_fp_fn_test[0]['acc']-s_attr_to_fp_fn_test[1]['acc'],"Diff_FPR":s_attr_to_fp_fn_test[0]['fpr']-s_attr_to_fp_fn_test[1]['fpr'],"Diff_FNR":s_attr_to_fp_fn_test[0]['fnr']-s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)

/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)




Accuracy: 0.6615384615384615




<ipython-input-18-b655b4ac98e7>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"O



Accuracy: 0.6615384615384615




<ipython-input-18-b655b4ac98e7>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"O

...

<ipython-input-18-b655b4ac98e7>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"O



Accuracy: 0.6473372781065089




<ipython-input-18-b655b4ac98e7>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"O



Accuracy: 0.6497041420118344




<ipython-input-18-b655b4ac98e7>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_W = summary_W.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[0]['acc'],"FPR":s_attr_to_fp_fn_test[0]['fpr'],"FNR":s_attr_to_fp_fn_test[0]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_B = summary_B.append({"Constraint_type":idx_to_constraint[i],"Accuracy":s_attr_to_fp_fn_test[1]['acc'],"FPR":s_attr_to_fp_fn_test[1]['fpr'],"FNR":s_attr_to_fp_fn_test[1]['fnr']},ignore_index=True)
<ipython-input-18-b655b4ac98e7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_diff = summary_diff.append({"Constraint_type":idx_to_constraint[i],"O

In [19]:
print("Summary for sensitive attribute value White (Caucasian):")
print(summary_W)

Summary for sensitive attribute value White (Caucasian):
     Constraint_type  Accuracy       FPR       FNR
0      No Constraint  0.657033  0.368421  0.323024
1  Misclassification  0.660886  0.372807  0.312715
2           Only FPR  0.630058  0.271930  0.446735
3           Only FNR  0.637765  0.333333  0.384880
4   Both FPR and FNR  0.641618  0.315789  0.391753


In [20]:
print("Summary for sensitive attribute value Black (African-American):")
print(summary_B)

Summary for sensitive attribute value Black (African-American):
     Constraint_type  Accuracy       FPR       FNR
0      No Constraint  0.668712  0.117318  0.591837
1  Misclassification  0.662577  0.162011  0.551020
2           Only FPR  0.674847  0.178771  0.503401
3           Only FNR  0.662577  0.223464  0.476190
4   Both FPR and FNR  0.662577  0.223464  0.476190


In [21]:
print("Summary of differences between sensitive attribute values:")
print(summary_diff)

Summary of differences between sensitive attribute values:
     Constraint_type  Overall Accuracy  Diff_Accuracy  Diff_FPR  Diff_FNR
0      No Constraint          0.661538      -0.011679  0.251103 -0.268813
1  Misclassification          0.661538      -0.001690  0.210796 -0.238306
2           Only FPR          0.647337      -0.044789  0.093159 -0.056666
3           Only FNR          0.647337      -0.024812  0.109870 -0.091311
4   Both FPR and FNR          0.649704      -0.020958  0.092326 -0.084438
